<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-Noble_Routines

## Authors: Leo Werneck, Zach Etienne, and Samuel Cupp

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the flavors of the Noble conservative-to-primitive routine available in GRHayL.

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

0. [Step 0](#src_dir): **Source directory creation**
1. [Step 1](#introduction): **Introduction**
1. [Step 2](#Noble_Routines): **`Noble Routines`**
    1. [Step 2.a](#variables_needed_by_Noble): *Setting up the variables needed by `Noble`*
        1. [Step 2.a.i](#Noble_vars): Conversion to Noble variables
        1. [Step 2.a.ii](#Noble_B): Computing $B^{i}_{\rm Noble}$
        1. [Step 2.a.iii](#Noble_prims): Writing $\boldsymbol{P}_{\rm Noble}$ in terms of $\boldsymbol{P}_{\rm GRHayL}$
    1. [Step 2.b](#newton_rapson): *Newton-Rapson Method*
    1. [Step 2.c](#returned_prims): *Returned values from `Noble` Routines*
        1. [Step 2.c.ii](#returned_v): Computing $v^i$
        1. [Step 2.c.iii](#variable_setup__conservs): Conservatives
1. [Step 3](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

GRHayL provides a variety of conservative-to-primitive algorithms. Of these, there are 4 different `Noble` routines TODO citation. All the `Noble` routines use the Newton-Rapson method to find values from which we can reconstruct the primitives. All the Noble routines compute the quantity
$$Z\equiv \frac{W^2 \tilde{\rho} h}{\psi^6}$$
where $W$ is the Lorentz factor, $\tilde{\rho}$ is the density conservative variable used by `GRHayL`, and $h$ is the enthalpy. The `Noble2D` routine also computes the squared 4-velocity $u^2$.

For more information on the arguments for conservative-to-primitive routines, see the tutorial for Hybrid_Multi_Method TODO link.

<a id='Noble_Routines'></a>

# Step 2: `Noble Routines` \[Back to [top](#toc)\]
$$\label{Noble_Routines}$$

<a id='variables_needed_by_Noble'></a>

## Step 2.a: Setting up the variables needed by `Noble` \[Back to [top](#toc)\]
$$\label{variables_needed_by_Noble}$$

Since the `Noble` routines are adapted from the [`HARM` software](https://arxiv.org/abs/astro-ph/0301509), they use a different set of conservative and primitive variables than the rest of `GRHayL`.

`GRHayL` uses the set of conservative variables
$\boldsymbol{C}_{\rm G} = \left\{\tilde{\rho},\tilde{\tau},\tilde{S}_{i},\tilde{B}^{i}\right\}$
and the primitive variables
$\boldsymbol{P}_{\rm G} = \left\{\rho_{b},P,v^{i},B^{i}\right\}$,
while `HARM` makes use of the conservative variables
$\boldsymbol{C}_{\rm HARM} = \left\{\tilde{\rho},\sqrt{-g}T^{0}_{\ 0}+\tilde{\rho},\tilde{S}_{i},\sqrt{-g}B^{i}_{\rm HARM}\right\}$
and primitive variables
$\boldsymbol{P}_{\rm HARM} = \left\{\rho_{b},u,\tilde{u}^{i},B^{i}_{\rm HARM}\right\}$
. The tilde quantities are defined in [Duez *et al.* (2005)](https://arxiv.org/pdf/astro-ph/0503420.pdf)
\begin{align}
\tilde{\rho} &= \sqrt{-g}\rho_{b}u^{0} \\
\tilde{\tau} &= \alpha\sqrt{-g}T^{00} - \tilde{\rho} \\
\tilde{S}_{i} &= \sqrt{-g}T^{0}_{\hphantom{0}i}
\end{align}
$\tilde{\rho}$ is defined in the text below Eq. 34, $\tilde{S}_{i}$ is defined in Eq. 36, and $\tilde{\tau}$ is defined in Eq. 37.

While we could convert to the `HARM` variables and then use the Noble routines, this involves extra steps because the `Noble` routines use *another* set of variables. It is more reasonable and less confusing to rework the outer layer of the `HARM` code to directly convert from `GRHayL` to `Noble` variables rather than converting twice.

Thus, the key step is then writing `Noble`'s variables in terms of `GRHayL`'s variables.

<a id='Noble_vars'></a>

### Step 2.a.i: Conversion to Noble variables \[Back to [top](#toc)\]
$$\label{Noble_vars}$$

Most of the conservative variables are related to the `Noble` variables through a simple relation. `Noble` uses undensitized variables, while `GRHayL` uses densitized variables. Thus, the density and momentum variables are related by

$$
\boxed{
\begin{align}
\rho^{\rm Noble} = \frac{\tilde{\rho}}{\psi^{6}}\\
S^{\rm Noble}_i = \frac{\tilde{S}_i}{\psi^{6}}
\end{align}
}
$$

where

$$\frac{1}{\psi^{6}} = \frac{\alpha}{\sqrt{-g}}$$

The more complicated relationship is for $\tilde{\tau}$, so we first convert from `GRHayL` to `HARM`. To do so, let us relate the `GRHayL` variables to the stress-energy tensor. First, consider the identity

$$
\begin{align}
S^{i} &= -\gamma^{i}_{\ \mu}n_{\nu}T^{\mu\nu}\\
      &= -\left(\delta^{i}_{\mu} + n^{i}n_{\mu}\right)n_{\nu}T^{\mu\nu}\\
      &= -\delta^{i}_{\mu}n_{\nu}T^{\mu\nu} - n^{i}n_{\mu}n_{\nu}T^{\mu\nu}\\
      &= -n_{0}T^{i0} - n^{i}n_{0}n_{0}T^{00}\\
      &= \alpha T^{i0} - \left(-\frac{\beta^{i}}{\alpha}\right)\alpha^{2}T^{00}\\
\implies S^{i}&= \alpha\left[\beta^{i}T^{00} + T^{i0}\right]
\end{align}
$$

Consider, also, the identity

$$
\tilde{\tau} = \alpha^{2}\sqrt{\gamma}T^{00} - \tilde{\rho} \implies T^{00} = \frac{\left(\tilde\tau + \tilde{\rho}\right)}{\alpha^{2}\sqrt{\gamma}}
$$

Then

$$
\begin{align}
T^{0}_{0} &= g_{0\mu}T^{0\mu}\\
          &= g_{00}T^{00} + g_{0i}T^{0i}\\
          &= \left(-\alpha^{2}+\beta_{\ell}\beta^{\ell}\right)T^{00} + \beta_{i}T^{0i}\\
          &= -\alpha^{2}T^{00} + \beta_{i}\left[\beta^{i}T^{00} + T^{i0}\right]\\
          &= -\alpha^{2}\left[\frac{\left(\tilde\tau + \tilde{\rho}\right)}{\alpha^{2}\sqrt{\gamma}}\right] + \frac{\beta^{i}S_{i}}{\alpha}\\
\implies T^{0}_{0} &= -\frac{\tilde\tau+\tilde{\rho}}{\sqrt{\gamma}} + \frac{\beta^{i}S_{i}}{\alpha}
\end{align}
$$

Using these identities, we can compute the relationship between $\tilde{\tau}_{\rm HARM}$ and $\tilde{\tau}$

$$
\begin{align}
\tilde\tau_{\rm HARM} &= \sqrt{-g}\left(T^{0}_{0} + \rho_{b}u^{0}\right)\\
%
&= \alpha\sqrt{\gamma}\left[-\frac{\tilde\tau+\tilde{\rho}}{\sqrt{\gamma}} + \frac{\beta^{i}S_{i}}{\alpha}\right] + \sqrt{-g}\rho_{b}u^{0}\\
%
&= -\alpha\tilde{\tau} -\alpha\tilde{\rho} + \beta^{i}\left(\sqrt{\gamma}S_{i}\right) + \tilde{\rho}\\
\implies &\boxed{\tilde\tau_{\rm HARM} = -\alpha\tilde{\tau} - \left(\alpha-1\right)\tilde{\rho} + \beta^{i}\tilde{S}_{i}}
\end{align}
$$

The `Noble` energy variable
$$u^{\rm Noble} = \frac{\tilde\tau_{\rm HARM} - \tilde{\rho}}{\psi^{6}}$$
is related to the `GRHayL` energy variable by
$$
\begin{align}
u^{\rm Noble} &= \frac{-\alpha\tilde{\tau} - \left(\alpha-1\right)\tilde{\rho} + \beta^{i}\tilde{S}_{i} - \tilde{\rho}}{\psi^{6}} \\
&= -\frac{\alpha\tilde{\tau}}{\psi^{6}} - \alpha\rho^{\rm Noble} + \beta^{i}S^{\rm Noble}_{i}
\end{align}
$$
If we simply define
$$
\tau^{\rm Noble} = \frac{\tilde{\tau}}{\psi^{6}}
$$
then
$$
\boxed{
u^{\rm Noble} = -\alpha\left(\tau^{\rm Noble} + \rho^{\rm Noble}\right) + \beta^{i}S^{\rm Noble}_{i}
}
$$

<a id='Noble_B'></a>

### Step 2.a.ii: Computing $B^{i}_{\rm Noble}$ \[Back to [top](#toc)\]
$$\label{Noble_B}$$

Finally, we must relate `Noble`'s $B^{i}_{\rm HARM}$ with the variables used by `GRHayL`. We can start by looking at eqs. (23), (24), and (31) in [Duez *et al.* (2005)](https://arxiv.org/pdf/astro-ph/0503420.pdf) to find the following useful relations

$$
{\rm GRHayL:}\ 
\left\{
\begin{align}
\sqrt{4\pi}b^{0} &= \frac{u_{i}B^{i}}{\alpha}\\
\sqrt{4\pi}b^{i} &= \frac{B^{i}/\alpha + \sqrt{4\pi}b^{0}u^{i}}{u^{0}}
\end{align}
\right.
$$

Now, if we look at eqs. (16) and (17) in [Gammie & McKinney (2003)](https://arxiv.org/pdf/astro-ph/0301509.pdf) we find the following relations

$$
{\rm HARM:}\ 
\left\{
\begin{align}
b^{0} &= u_{i}B^{i}_{\rm HARM}\\
b^{i} &= \frac{B^{i}_{\rm HARM} + b^{0}u^{i}}{u^{0}}
\end{align}
\right.
$$

The conversion between `HARM` and `Noble` is given by

$$
B^{i}_{\rm Noble} = \alpha B^{i}_{\rm HARM}
$$

Then,
$$
B^{i}_{\rm Noble} = \frac{B^{i}}{\sqrt{4\pi}}
$$

Summarizing the previous steps, we simply need to compute the undensitized variables from the `GRHayL` variables, which we now label as "undens" and compute the `Noble` variables using the relations

$$
\begin{align}
\rho^{\rm Noble} &= \rho^{\rm undens} \\
u^{\rm Noble} &= -\alpha\left(\tau^{\rm undens} + \rho^{\rm undens}\right) + \beta^{i}S^{\rm undens}_{i} \\
S^{\rm Noble}_i &= S_i^{\rm undens} \\
B^{i}_{\rm Noble} &= \frac{B^{i}}{\sqrt{4\pi}}
\end{align}
$$
Note that only the primitive_quantities struct has a $B^i$ variable, which is the undensitized variable. This is because the conservative-to-primitive routine doesn't need to compute this primitive, so it can be given and used directly instead of using the conservative $\tilde{B}^i$.

<a id='Noble_prims'></a>

### Step 2.a.iii: Writing $\boldsymbol{P}_{\rm Noble}$ in terms of $\boldsymbol{P}_{\rm GRHayL}$ \[Back to [top](#toc)\]
$$\label{Noble_prims}$$

Keep in mind that for the primitive variables, we are not interested in providing exact values. After all, the primitives are the unknowns we are trying to find. Instead, what we are interested in doing is providing an educated guess of what it should look like in hopes that the Newton-Raphson method then converges to the correct values. The details of what the initial guesses are in `GRHayL` variables are discussed in general within the `Hybrid_Multi_Method` tutorial
TODO link
. To convert to the Noble variables,

$$
\boxed{
\begin{align}
\rho_b^{\rm Noble} &= \rho_{b}\\
u &= \frac{P}{\Gamma_{\rm poly} - 1}\\
\tilde{u}^{i} &= u^{0}\left(v^{i}+\beta^{i}\right)\\
B^{i}_{\rm Noble} &= \frac{B^{i}}{\sqrt{4\pi}}
\end{align}\ ,
}
$$

where $\Gamma_{\rm poly}$ stands for the *local* polytropic $\Gamma$-factor.

<a id='newton_rapson'></a>

## Step 3.b: Newton-Rapson Method \[Back to [top](#toc)\]
$$\label{newton_rapson}$$

fff

<a id='returned_prims'></a>

## Step 3.c: Returned values from `Noble` Routines \[Back to [top](#toc)\]
$$\label{returned_prims}$$

Once the `Newton-Rapson` method converges (or fails), it returns to the `Noble` routines with an error code. We will combine these error codes with those from `Noble` at the end of this Step. As mentioned in the [Introduction](#introduction), all routines return the quantity $Z$, and the `Noble2D` routine also returns the squared 4-velocity magnitude $u^2$. The various `Noble1D` routines instead compute $u^2$ from $Z$. If
$$u^2>1$$
it is set to
$$1-2*10^{-16}$$
Then, we begin computing the primitives. The baryonic density is
$$
\rho_b = \frac{\rho^{\rm undens}}{W}
$$
where the Lorenz factor $W=\left(1-u^2\right)^{-1/2}$.

<a id='returned_v'></a>

### Step 3.c.i: Computing $v^i$ \[Back to [top](#toc)\]
$$\label{returned_v}$$

To compute the velocities, we compute the `HARM` variable $\tilde{u}^i$ and then compute $v^i$ from it. The full expression for $\tilde{u}^i$ is
$$
\tilde{u}^i = \frac{W}{Z+B^2}\left(\tilde{Q}^i + \frac{Q_j B^j}{Z} B^i \right)
$$
This is, unfortunately, going to have several definitions of variables in a row. These intermediate quantities are either used inside the `Newton-Rapson` method or are used to compute *other* intermediate quantities which are used in the `Newton-Rapson` method. We define them all here so that they are as close to the $\tilde{u}^i$ equation as possible. $Q^i$ and $\tilde{Q}^i$ are given by
\begin{align}
\tilde{Q}^i &\equiv Q^i + n^i Q^\mu n_\mu \\
Q^\mu &= g^{\mu\nu}Q_{\mu\nu} \\
Q_0 &\equiv u^{\rm Noble} \\
Q_i &\equiv S_i^{\rm Noble} \\
n_\mu &= -\alpha \delta^0_\mu \\
n^\mu &= -\frac{1}{\alpha}\{1, \beta^i\}
\end{align}
where $n_\mu$ is the normal vector to the spatial hypersurface.
For completeness, we rework the $\tilde{u}^i$ equation to be in terms of the input variables
\begin{align}
\tilde{u}^i &= \frac{W}{Z+B^2}\left(\tilde{Q}^i + \frac{Q_j B^j}{Z} B^i \right) \\
%
&= \frac{W}{Z+B^2}\left(Q^i + n^i Q^j n_j + \frac{S_j^{\rm Noble} B^j}{Z} B^i \right) \\
%
&= \frac{W}{Z+B^2}\left(g^{i\nu}Q_{\nu} - \alpha n^i Q^0\right) + \frac{W}{Z^2+B^2 Z}\left(S_j^{\rm Noble} B^j B^i \right) \\
%
&= \frac{W}{Z+B^2}\left(g^{0i}Q_{0} + g^{ij}Q_{j} + \alpha \frac{\beta^i}{\alpha} u^{\rm Noble}\right) + \frac{W}{Z^2+B^2 Z}\left(S_j^{\rm Noble} B^j B^i \right) \\
%
&= \frac{W}{Z+B^2}\left(\beta^i u^{\rm Noble} + \gamma^{ij}S_{j}^{\rm Noble} + \beta^i u^{\rm Noble}\right) + \frac{W}{Z^2+B^2 Z}\left(S_j^{\rm Noble} B^j B^i \right) \\
%
&= u^{\rm Noble}\frac{2\beta^i W}{Z+B^2} + \frac{S_{j}^{\rm Noble}W}{Z+B^2} \left(\gamma^{ij} + \frac{B^j B^i}{Z} \right) \\
\end{align}
As is hopefully obvious, this equation is significantly more work to compute, especially since $Q^\mu$, $Q_\mu$, and $Q^\mu n_\mu$ are already computed.

After computing $\tilde{u}^i$, we call the `limit_utilde_and_compute_v` function (same limit method as in [limit_v_and_compute_u0](../Tutorial-limit_v_and_output_u0.ipynb)) to apply a speed limit to $\tilde{u}^i$ before computing
$$
v^i = \frac{\tilde{u}^i}{u^0} - \beta^i
$$
where $u^0$ is the 0th component of the 4-velocity. If the speed is limited, then $\rho_b$ needs to be recomputed as well
$$
\rho_b = \frac{\rho^{\rm undens}}{\sqrt{-g} u^0} = \frac{\rho^{\rm undens}}{\alpha \psi^6 u^0}
$$

<a id='returned_other'></a>

### Step 3.c.ii: Computing remaining primitives \[Back to [top](#toc)\]
$$\label{returned_other}$$

Now that we have computed $v^i$ and $\rho_b$, we can compute the remaining primitive variables. Since $\rho_b$ potentially changed due to the speed limiter, we recalculate `Noble`'s energy variable $u$ using the EOS and $\rho_b$:
$$
u = \frac{P_{\rm cold}}{\Gamma_{\rm ppoly}-1}
$$
where
$$
P_{\rm cold} = K_{\rm ppoly}\left(\rho_b\right)^{\Gamma_{\rm ppoly}}
$$
and $K_{\rm ppoly}$ and $\Gamma_{\rm ppoly}$ are the local polytropic values obtained from the EOS function `hybrid_get_K_and_Gamma`. Then, we compute the pressure using
$$
P = P_{\rm cold} + (\Gamma_{th} - 1)*(u - \rho_b*\epsilon_{\rm cold})
$$
which is provided by `pressure_rho0_u` in the (TODO currently not accessible through grhayl) harm header. Then,
$$
\epsilon = \frac{u}{\rho_b}
$$
and compute the entropy using `hybrid_compute_entropy_function` if evolving entropy.

<a id='return_val'></a>

### Step 3.c.iii: Returned value \[Back to [top](#toc)\]
$$\label{return_val}$$

The `Hybrid_Noble2D` function returns a value depending on the results of the `Newton-Rapson` method. There are The possible return values are

| Return Value | Meaning
|--------------|:-----------
| 0            | C2P routine successfully computed primitives
| 2            | Initial primitives guess makes $\tilde{u}$ either negative or too large
| 3            | $Z<0$ or $Z>Z\_TOO\_BIG$
| 4            | $u^2 < 0$
| 5            | $\rho_b \le 0.0$ or $u \le 0.0$
| 101          | `Newton-Raphson` solver did not converge to a solution with the given tolerances
| 201          | `Newton-Raphson` procedure encountered a numerical divergence (occurrence of "nan" or "+/-inf")

The only non-zero error code which does not immediately return is error code 5. While these are invalid values, the actual `Newton-Raphson` solver converged to acceptible values. Some might choose to simply apply floors and ceilings to the primitives for this error code instead of considering return value 5 a failure, so the primitives are also computed for this case. Please note that these do not line up exactly with the original `HARM` error codes, so for codes which are transitioning to use `GRHayL` it is important to double check that the code will match the intended behavior.

<a id='latex_pdf_output'></a>

# Step 3: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__driver_conserv_to_prims.pdf](Tutorial-IllinoisGRMHD__driver_conserv_to_prims.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).